In [2]:
import yaml
from pydantic import BaseModel, Field
from typing import Optional, Dict, Any
from enum import Enum
import importlib

# Step 1: Lazy Loading Helper Function
def lazy_load(module_name: str, class_name: str):
    try:
        # Dynamically import the module
        module = importlib.import_module(module_name)
        # Get the class from the module
        return getattr(module, class_name)
    except Exception as e:
        raise ValueError(f"Error loading {class_name} from module {module_name}: {e}")

# Step 2: Enum Class for LLM Types
class LLM(str, Enum):
    OPENAI = "openai"
    AZURE_OPENAI = "azure_openai"
    HUGGINGFACE = "huggingface"
    OLLAMA = "ollama"
    COHERE = "cohere"
    VERTEXAI = "vertexai"
    BEDROCK = "bedrock"
    JINA = "jina"
    CUSTOM = "custom"

# Step 3: Map LLM Types to Lazy-loaded Embedding Classes
LLM_MAP = {
    LLM.OPENAI: lazy_load("langchain_openai", "ChatOpenAI"),
    LLM.AZURE_OPENAI: lazy_load("langchain_openai", "AzureChatOpenAI"),
}

# Step 4: Define the LLM Configuration Model
class LLMConfig(BaseModel):
    model_config = {"protected_namespaces": ()}
    
    type: LLM  # Enum to specify the LLM
    model_kwargs: Optional[Dict[str, Any]] = Field(default_factory=dict, description="Model-specific parameters like model name/type")
    custom_class: Optional[str] = None  # Optional: If using a custom class

In [3]:
from typing import List, Union
from pydantic_v1 import Field, BaseModel


class BaseConfig(BaseModel):
    """Base configuration shared across all RAG modules"""
    # input_source: Union[str, List[str]] = Field(..., description="File path, directory path, or URL for input data")
    # test_dataset: str = Field(..., description="Path to CSV file containing test questions")
    
    @classmethod
    def from_yaml(cls, file_path: str) -> "GenerationOptionsConfig":
        with open(file_path, "r") as yaml_file:
            config = yaml.safe_load(yaml_file)
        return cls(**config["Generation"])

    def to_yaml(self, file_path: str) -> None:
        """Save configuration to a YAML file."""
        with open(file_path, 'w') as file:
            yaml.dump(self.model_dump(), file)

# Step 2: Define Pydantic Model for Individual LLM Configuration
class GenerationConfig(BaseConfig):
    type: LLM  # Specifies the LLM type
    model_kwargs: Optional[Dict[str, Any]] = Field(default_factory=dict, description="Model-specific parameters")
    prompt_template: Optional[str] = None

# Step 3: Define Pydantic Model for Overall Generation Configuration
class GenerationOptionsConfig(BaseConfig):
    llms: List[GenerationConfig]  # List of LLM configurations
    prompt_template_path: Optional[str] = None


In [4]:
import yaml
import os
import requests
from ragbuilder.generation.config import PromptTemplate
import pandas as pd
def load_prompts(file_name: str = "rag_prompts.yaml", url: str= os.getenv("RAG_PROMPT_URL"),read_local: bool = False):
    """
    Load YAML prompts either from a local file or an online source.

    Args:
        file_name (str): Name of the YAML file. Defaults to "rag_prompts.yaml".
        read_local (bool): If True, read from a local file. Otherwise, fetch from an online URL.

    Returns:
        List[PromptTemplate]: A list of PromptTemplate objects.
    """
    yaml_content = None

    if read_local:
        # Attempt to read from the local file
        if os.path.exists(file_name):
            print(f"Loading prompts from local file: {file_name}")
            with open(file_name, 'r') as f:
                yaml_content = f.read()
        else:
            raise FileNotFoundError(f"Local file not found: {file_name}")
    else:
        # Attempt to fetch from an online source
        print(f"Fetching prompts from online file: {url}")
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an HTTP error for bad responses
            yaml_content = response.text
        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"Failed to load prompts from URL {url}: {e}")

    # Parse the YAML content
    try:
        prompts_data = yaml.safe_load(yaml_content)
    except yaml.YAMLError as e:
        raise ValueError(f"Failed to parse YAML content: {e}")

    # Convert YAML entries into PromptTemplate objects
    prompts = [
        PromptTemplate(name=entry['name'], template=entry['template'])
        for entry in prompts_data
    ]
    return prompts

/Users/ashwinaravind/.pyenv/versions/3.12.5/lib/python3.12/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import chromadb

from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.retrievers import EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from ragbuilder.generation.evaluation import RAGASEvaluator
def sample_retriever():
    print("rag_get_retriever initiated")
    try:
        def format_docs(docs):
            return "\n".join(doc.page_content for doc in docs)

        # LLM setup
        llm = AzureChatOpenAI(model="gpt-4o-mini")

        # Document loader
        loader = WebBaseLoader("https://raw.githubusercontent.com/ashwinaravind/ashwinaravind.github.io/refs/heads/main/thevanishingtown")
        docs = loader.load()

        # Embedding model
        embedding = AzureOpenAIEmbeddings(model="text-embedding-3-large")

        # Text splitting and embedding storage
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
        splits = splitter.split_documents(docs)

        # Initialize Chroma database
        c = Chroma.from_documents(
            documents=splits,
            embedding=embedding,
            collection_name="testindex-ragbuilder-retreiver",
            client_settings=chromadb.config.Settings(allow_reset=True),
        )

        # Retriever setup
        retriever = c.as_retriever(search_type="similarity", search_kwargs={"k": 5})
        ensemble_retriever = EnsembleRetriever(retrievers=[retriever])
        print("rag_get_retriever completed")
        return ensemble_retriever
    except Exception as e:
        import traceback
        print(f"An error occurred: {e}")
        traceback.print_exc()
        return None
# sample_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# Step 5: Load YAML File and Parse Configurations
from typing import List, Dict, Type
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda


from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
class SystemPromptGenerator:
    def __init__(self, config: GenerationOptionsConfig, evaluator_class: Type):
        self.llms = []  # List to store instantiated LLMs
        self.config = config
        print("printing config",config)
        # self.evaluator = evaluator_class() 
        self.retriever=sample_retriever
        if config.prompt_template_path:
            self.prompt_templates = load_prompts(config.prompt_template_path)
        else:
            self.prompt_templates = load_prompts()
        # for llm_config in config.llms:
        #     llm_class = LLM_MAP[llm_config.type]  # Get the corresponding LLM class)
    def _build_trial_config(self) -> List[GenerationConfig]:
            """
            Build a list of GenerationConfig objects from the provided GenerationOptionsConfig.

            Args:
                options_config (GenerationOptionsConfig): The input configuration for trial generation.

            Returns:
                List[GenerationConfig]: A list of generated configurations for trials.
            """
            trial_configs = []
            for llm_config in self.config.llms:
                # llm_class = LLM_MAP[llm_config.type]  # Get the corresponding LLM class)
                llm_instance = LLMConfig(type=llm_config.type, model_kwargs=llm_config.model_kwargs)
                llm_class = LLM_MAP[llm_config.type]
                # Step 8: Instantiate the Model with the Configured Parameters
                llm = llm_class(**llm_config.model_kwargs)
                # print(llm_config.type,llm_config.model_kwargs,llm.invoke("what is the capital of France?"))
                # print(self.prompt_templates)
                for prompt_template in self.prompt_templates:
                    trial_config = GenerationConfig(
                        type=llm_config.type,  # Pass the LLMConfig instance here
                        model_kwargs=llm_config.model_kwargs,
                        # evaluator=self.evaluator,
                        # retriever=self.retriever,
                        # eval_data_set_path=self.config.eval_data_set_path,
                        prompt_template=prompt_template.template,
                        # read_local_only=self.config.read_local_only,
                        )
                    # res=self._create_pipeline(trial_config,self.retriever()).invoke("Who is Clara?")
                    # print(res)
                    trial_configs.append(trial_config)
                    break

                # print(trial_config)
                # trial_configs.append(trial_config)

            # print(trial_configs)
            # for llm_config in options_config.llms:
            #     print(llm_config)
            #     trial_config = GenerationConfig(
            #         llm_type=llm_config.type,
            #         llm_model_kwargs=llm_config.model_kwargs,
            #         evaluator=options_config.evaluator,
            #         retriever=options_config.retriever,
            #         eval_data_set_path=options_config.eval_data_set_path,
            #         prompt_template_path=options_config.prompt_template_path,
            #         read_local_only=options_config.read_local_only,
            #     )
            #     trial_configs.append(trial_config)
            return trial_configs
    def _create_pipeline(self, trial_config: GenerationConfig, retriever: RunnableParallel):
        try:
            def format_docs(docs):
                return "\n".join(doc.page_content for doc in docs)

            # Prompt setup
            llm_class = LLM_MAP[trial_config.type]
                # Step 8: Instantiate the Model with the Configured Parameters
            llm = llm_class(**trial_config.model_kwargs)
            prompt_template = trial_config.prompt_template
            print('prompt_template',prompt_template)
            print("testing retriever\n",retriever.invoke("Who is Clara?"))
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", prompt_template),
                    ("user", "{question}"),
                    MessagesPlaceholder(variable_name="chat_history", optional=True),
                ]
            )

            # RAG Chain setup
            rag_chain = (
                RunnableParallel(context=retriever, question=RunnablePassthrough())
                .assign(context=itemgetter("context") | RunnableLambda(format_docs))
                .assign(answer=prompt | llm | StrOutputParser())
                .pick(["answer", "context"])
            )
            print("rag_pipeline completed")
            return rag_chain
        except Exception as e:
            import traceback
            print(f"An error occurred: {e}")
            traceback.print_exc()
            return None
    def optimize(self):
        trial_configs = self._build_trial_config()
        print("trial_configs",trial_configs)
        pipeline=None
        results = {}
        evaluator = RAGASEvaluator()
        evaldataset=evaluator.get_eval_dataset('/Users/ashwinaravind/Desktop/kruxgitrepo/ragbuilder/gensimtest.csv')
        for trial_config in trial_configs:
            pipeline = self._create_pipeline(trial_config,self.retriever())
            for entry in evaldataset:
                question = entry.get("question", "")
                result=pipeline.invoke(question)
                results[trial_config.prompt_template] = []
                results[trial_config.prompt_template].append({
                        "prompt_key": trial_config.prompt_template,
                        "prompt": trial_config.prompt_template,
                        "question": question,
                        "answer": result.get("answer", "Error"),
                        "context": result.get("context", "Error"),
                        "ground_truth": entry.get("ground_truth", ""),
                        "config": trial_config.dict(),
                    })
                break
        output_data = []
        for prompt_key, prompt_results in results.items():
            output_data.extend(prompt_results)

        # Convert to a Dataset directly
        from datasets import Dataset
        results_dataset = Dataset.from_list(output_data)

        # Optionally clean up or format the dataset
        if "context" in results_dataset.column_names:
            results_dataset = results_dataset.map(
                lambda x: {
                    **x,
                    "contexts": eval(x["context"]) if isinstance(x["context"], str) and x["context"].startswith("[") else [x["context"]],
                }
            )

        print("test_prompt completed")
        # return results_dataset
        eval_results=evaluator.evaluate(results_dataset)
        # print('eval_results',GenerationConfig(**eval_results['config'][0]))
        final_results=self.calculate_metrics(eval_results)
        return final_results
    # def calculate_metrics(self, result):
    #     # Convert the results to a pandas DataFrame
    #     print("writing config",result['config'])
    #     results_df = result.to_pandas()
        
    #     # Calculate average correctness per prompt key
    #     average_correctness = results_df.groupby(['prompt_key','prompt'])['answer_correctness',].mean().reset_index()
    #     average_correctness.columns = ['prompt_key', "prompt", 'average_correctness']
        
    #     # Save the average correctness to a CSV file
    #     average_correctness.to_csv('rag_average_correctness.csv', index=False)
    #     print("The average correctness results have been saved to 'rag_average_correctness.csv'")
        
    #     # Find the row with the highest average correctness
    #     best_prompt_row = average_correctness.loc[average_correctness['average_correctness'].idxmax()]
        
    #     # Extract prompt_key, prompt, and average_correctness
    #     prompt_key = best_prompt_row['prompt_key']
    #     prompt = best_prompt_row['prompt']
    #     max_average_correctness = best_prompt_row['average_correctness']
    #     config=best_prompt_row['config']
    #     return prompt_key, prompt, max_average_correctness,config
    #     return config
    def calculate_metrics(self, result):
    # Convert the results to a pandas DataFrame
        results_df = result.to_pandas()

        # Group by `prompt_key` and calculate average correctness, while retaining `prompt` and `config`
        grouped_results = (
            results_df.groupby('prompt_key')
            .agg(
                prompt=('prompt', 'first'),  # Take the first prompt for each prompt_key
                config=('config', 'first'),  # Take the first config for each prompt_key
                average_correctness=('answer_correctness', 'mean')  # Calculate average correctness
            )
            .reset_index()
        )

        # Save the results to a CSV file
        grouped_results.to_csv('rag_average_correctness.csv', index=False)
        print("The average correctness results have been saved to 'rag_average_correctness.csv'")

        # Find the row with the highest average correctness
        best_prompt_row = grouped_results.loc[grouped_results['average_correctness'].idxmax()]

        # Extract prompt_key, prompt, average_correctness, and config
        prompt_key = best_prompt_row['prompt_key']
        prompt = best_prompt_row['prompt']
        max_average_correctness = best_prompt_row['average_correctness']
        config = best_prompt_row['config']

        return prompt_key, prompt, max_average_correctness, config

        # return prompt_key, prompt, max_average_correctness

In [7]:
#how to use
gen=SystemPromptGenerator(GenerationOptionsConfig.from_yaml("config.yaml"),RAGASEvaluator)
prompt_key, prompt, max_average_correctness, config=gen.optimize()
best_config_py_model=GenerationConfig(**config)
best_gen=gen._create_pipeline(best_config_py_model,sample_retriever())
best_gen.invoke("Who is Clara?")

printing config llms=[GenerationConfig(type=<LLM.AZURE_OPENAI: 'azure_openai'>, model_kwargs={'model_name': 'gpt-4o-mini', 'temperature': 0.6}, prompt_template=None)] prompt_template_path=None
Fetching prompts from online file: https://raw.githubusercontent.com/ashwinaravind/rag_prompts/refs/heads/main/rag_prompts.yml
trial_configs [GenerationConfig(type=<LLM.AZURE_OPENAI: 'azure_openai'>, model_kwargs={'model_name': 'gpt-4o-mini', 'temperature': 0.6}, prompt_template='You are a helpful assistant. Answer any questions solely based on the context provided below. \nIf the provided context does not have the relevant facts to answer the question, say "I don\'t know."\n\n<context>\n{context}\n</context>\n')]
RAGASEvaluator initiated
rag_get_retriever initiated
rag_get_retriever completed
prompt_template You are a helpful assistant. Answer any questions solely based on the context provided below. 
If the provided context does not have the relevant facts to answer the question, say "I don't k

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

test_prompt completed


Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

evaluate_prompts completed
Dataset({
    features: ['prompt_key', 'prompt', 'question', 'answer', 'ground_truth', 'answer_correctness', 'faithfulness', 'answer_relevancy', 'context_precision', 'context_recall', 'config'],
    num_rows: 1
})
The average correctness results have been saved to 'rag_average_correctness.csv'
rag_get_retriever initiated
rag_get_retriever completed
prompt_template You are a helpful assistant. Answer any questions solely based on the context provided below. 
If the provided context does not have the relevant facts to answer the question, say "I don't know."

<context>
{context}
</context>

testing retriever
 [Document(metadata={'source': 'https://raw.githubusercontent.com/ashwinaravind/ashwinaravind.github.io/refs/heads/main/thevanishingtown'}, page_content='A young woman.\n\n---\n\n## Chunk 3: The Locket’s Secret\n\nClara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His gr

{'answer': "Clara is a young woman who wakes up in a cabin and has a mysterious encounter with a man who claims the cabin is his family's. She feels a sense of familiarity with him, despite believing they have never met before.",
 'context': 'A young woman.\n\n---\n\n## Chunk 3: The Locket’s Secret\n\nClara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.\n\n“Who are you?” she stammered, clutching her locket.\n\n“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”\n“Who are you?” she stammered, clutching her locket.\n\n“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”\n\nClara’s mind raced. Family? She looked him over. There was something familiar about him, though she was certain they had never met. But those eyes—they felt as though t